# Hands-on lab for Tallinn Data Week 

Welcome to this hands-on workshop, which is part of the [Tallinn Data Week](https://fienta.com/tallinn-data-week-2025-snowflake) program. The goal of this workshop is to give you practical experience with Snowflake by combining concepts from modern data engineering pipelines with elements of data processing using Modin and Cortex.

Before we start, please make sure you have a Snowflake trial account set up: https://signup.snowflake.com 

When signing up, select the cloud provider **AWS** and a region **EU (Frankfurt)**. This ensures faster performance and better alignment with the resources you’ll use in the workshop.

Once your account is ready, we will walk through how to import a Jupyter notebook directly in the Snowflake UI. You can also see the steps from here:

1. Download the [notebook](https://github.com/solita/tallinn-data-week-snowflake/blob/main/TDW_SNOWFLAKEDATAENGINEERING.ipynb)
2. Log in to your Snowflake UI.
3. Click the + Create button on the left sidebar.
4. Choose Notebook → Import .ipynb File
4. For database, select SNOWFLAKE_LEARNING_DB.
5. For schema, select PUBLIC.
6. Keep the runtime and runtime warehouse as the default.
7. Keep query warehouse as COMPUTE_WH and notebook warehouse as SYSTEM$STREAMLIT_NOTEBOOK_WH.
8. Finally, click Create to import the notebook into your environment.

The notebook will contain step-by-step instructions, this will allow you to follow along with the exercises and run the queries on your own environment.

## What This Workshop Covers

1. Data Engineering Foundations
- How to set up stages and file formats.
- Loading raw CSV data into Snowflake tables.
- Transforming and cleaning the data with SQL and views.
- Doing analytical queries
2. Extending with Modin and Cortex
- Applying Cortex features to enrich the dataset. It covers:
  - Classification function
  - Translation function
  - Summarisation function
  - Sentiment Analysis
- Using the GET_LINEAGE function to understand data dependencies and flows.
- Building a simple Streamlit app to demonstrate your results.


In [ ]:
use role sysadmin;
CREATE OR REPLACE DATABASE avalanche;
CREATE OR REPLACE SCHEMA raw;

Let's create a FILE FORMAT object. A FILE FORMAT describes a set of staged data.

In [ ]:
CREATE OR REPLACE FILE FORMAT avalanche.raw.csv_ff
  TYPE = 'CSV'
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  FIELD_DELIMITER = ','
  SKIP_HEADER = 1;

Next, let's create a STAGE. In Snowflake, a STAGE defines a location in cloud storage, where files are stored for loading (or unloading).

In [ ]:
-- create stage  
CREATE OR REPLACE STAGE avalanche.raw.avalanche
url = 's3://sfquickstarts/misc/avalanche/csv/'
file_format = avalanche.raw.csv_ff;

-- see what is in the stage
ls @avalanche;

We will create a couple of tables, PRODUCT_CATALOG and ORDER_HISTORY, and load them.

In [ ]:
-- product catalog table build
CREATE OR REPLACE TABLE AVALANCHE.RAW.PRODUCT_CATALOG
(
   NAME VARCHAR(16777216),
   DESCRIPTION VARCHAR(16777216),
   PRICE VARCHAR(16777216)
);

-- create table for order, more to analyze in here
CREATE OR REPLACE TABLE AVALANCHE.RAW.ORDER_HISTORY (
  ORDER_ID      VARCHAR(100),
  CUSTOMER_ID   VARCHAR(100),
  PRODUCT_ID    VARCHAR(100),
  PRODUCT_NAME  VARCHAR(16777216),
  QUANTITY_ORDERED INTEGER,
  PRICE         VARCHAR(16777216),
  TOTAL_PRICE   VARCHAR(16777216),
  ORDER_DATE    DATE
);

In [ ]:
SELECT t.$1, t.$2, t.$3  FROM @avalanche/product-catalog.csv (file_format => 'avalanche.raw.csv_ff') t;


In [ ]:
-- product catalog table load
COPY INTO avalanche.raw.product_catalog
FROM @avalanche/product-catalog.csv;

-- check the table
SELECT * FROM avalanche.raw.product_catalog;


In [ ]:
COPY INTO AVALANCHE.RAW.ORDER_HISTORY
FROM @avalanche/order-history.csv;

select * from AVALANCHE.RAW.ORDER_HISTORY;

Now that our raw data is loaded into Snowflake tables, let's do some transformations.

In [ ]:
-- create a fresh schema
CREATE OR REPLACE SCHEMA AVALANCHE.TRF;
USE SCHEMA AVALANCHE.TRF;

In the data we loaded, the PRICE column is a string, and it looks like a US price with a dollar sign. We would rather store this as a number, so that we can aggregate and analyze it. 

Let's create a UDF called CLEAN_PRICE to clean the PRICE column, and then use that UDF to create some views with cleaned data. 

In [ ]:
CREATE OR REPLACE FUNCTION TRF.CLEAN_PRICE(s STRING)
RETURNS NUMBER(18,2)
AS
$$
  TRY_TO_DECIMAL(REGEXP_REPLACE(s, '[^0-9\\.-]', ''), 18, 2)
$$;

In [ ]:

CREATE OR REPLACE VIEW PRODUCTS AS
SELECT
  NAME,
  DESCRIPTION,
  TRF.CLEAN_PRICE(PRICE) AS PRICE_NUM, -- remove $ and cast price to numeric
  -- also create groupings using basic string functions
  IFF(LOWER(NAME) LIKE '%ski%' OR LOWER(DESCRIPTION) LIKE '%ski%', 'ski', NULL) AS CAT_SKI,
  IFF(LOWER(NAME) LIKE '%boot%' OR LOWER(DESCRIPTION) LIKE '%boot%', 'boot', NULL) AS CAT_BOOT,
  IFF(LOWER(NAME) LIKE '%helmet%' OR LOWER(DESCRIPTION) LIKE '%helmet%', 'helmet', NULL) AS CAT_HELMET,
  IFF(LOWER(NAME) LIKE '%jacket%' OR LOWER(DESCRIPTION) LIKE '%jacket%', 'jacket', NULL) AS CAT_JACKET
FROM AVALANCHE.RAW.PRODUCT_CATALOG;

select * from PRODUCTS;

In [ ]:
CREATE OR REPLACE VIEW ORDERS AS
SELECT
  ORDER_ID,
  CUSTOMER_ID,
  PRODUCT_ID,
  PRODUCT_NAME,
  TRY_TO_NUMBER(QUANTITY_ORDERED) AS QTY, -- TRY_TO functions can be helpful when you just want to get what is valid
  TRF.CLEAN_PRICE(PRICE) AS UNIT_PRICE, -- cleaned price
-- now use both to make a review column
  COALESCE(
    TRF.CLEAN_PRICE(PRICE) * TRY_TO_NUMBER(QUANTITY_ORDERED),
    TRF.CLEAN_PRICE(PRICE)
  ) AS TOTAL_PRICE_NUM,
  TO_DATE(ORDER_DATE) AS ORDER_DATE
FROM AVALANCHE.RAW.ORDER_HISTORY;

select * from ORDERS;

Let's play around a bit and do some basic analyis. This is often the kinds of calculations you would want for reports.

In [ ]:
-- Revenue by month
SELECT DATE_TRUNC('month', ORDER_DATE) AS month, ROUND(SUM(TOTAL_PRICE_NUM),2) AS revenue
FROM ORDERS
GROUP BY 1
ORDER BY 1;

In [ ]:
-- Top products by revenue
SELECT 
    PRODUCT_NAME, 
    ROUND(SUM(TOTAL_PRICE_NUM),2) AS REVENUE, 
    SUM(QTY) AS QTY
FROM ORDERS
GROUP BY PRODUCT_NAME
ORDER BY REVENUE DESC;

In [ ]:
-- Price vs. demand 
SELECT 
    ROUND(UNIT_PRICE,2) AS PRICE, 
    SUM(QTY) AS QTY
FROM ORDERS
WHERE UNIT_PRICE IS NOT NULL
GROUP BY 1
ORDER BY 1;

In [ ]:
--  Create sales enriched view by combining orders and product category
CREATE OR REPLACE VIEW SALES_ENRICHED AS
SELECT
  o.ORDER_ID,
  o.CUSTOMER_ID,
  o.ORDER_DATE,
  o.PRODUCT_ID,
  o.PRODUCT_NAME,
  o.QTY,
  o.UNIT_PRICE,
  o.TOTAL_PRICE_NUM,
  p.PRICE_NUM AS CATALOG_PRICE
FROM ORDERS o
LEFT JOIN PRODUCTS p
  ON o.PRODUCT_NAME = p.NAME  
;

SELECT * FROM SALES_ENRICHED;

In [ ]:
-- Catalog Price vs. Order Price
SELECT PRODUCT_NAME,
       ROUND(AVG(UNIT_PRICE), 2)   AS avg_sold_price,
       ROUND(AVG(CATALOG_PRICE),2) AS avg_catalog_price,
       ROUND(SUM(TOTAL_PRICE_NUM),2) AS total_revenue
FROM SALES_ENRICHED
GROUP BY PRODUCT_NAME
ORDER BY total_revenue DESC;

Snowflake Cortex offers powerful AI capabilities that you can apply directly to your data, transforming raw text into structured insights. These functions are built-in and ready to use.

Let's classify our products using the AISQL function AI_CLASSIFY. The syntax is 

```sql
AI_CLASSIFY( <input> , <list_of_categories> [, <config_object>] )
```

In [ ]:
CREATE OR REPLACE TABLE PRODUCTS_CLASSIFY_1 AS
SELECT
  NAME,
  DESCRIPTION,
  PRICE_NUM, 
  AI_CLASSIFY(NAME, ['Apparel', 'Accessories']) as LABEL_1,
  AI_CLASSIFY(DESCRIPTION, ['Apparel', 'Accessories']) as LABEL_2,
FROM AVALANCHE.TRF.PRODUCTS;

-- check the data
SELECT * FROM PRODUCTS_CLASSIFY_1;

In [ ]:
-- extract the value from the JSON
CREATE OR REPLACE TABLE PRODUCTS_CLASSIFY AS
SELECT
  NAME,
  DESCRIPTION,
  PRICE_NUM, 
  LABEL_1:labels[0]::string as CATEGORY_1,
  LABEL_2:labels[0]::string as CATEGORY_2
FROM AVALANCHE.TRF.PRODUCTS_CLASSIFY_1;

select * from PRODUCTS_CLASSIFY;

Snowflake Cortex makes language translation simple. Let's leverage Cortex's powerful TRANSLATE function to instantly localize our product information from English into a couple of different languages:

In [ ]:
CREATE OR REPLACE TABLE PRODUCTS_TRANSLATE AS
SELECT
  NAME,
  SNOWFLAKE.CORTEX.TRANSLATE(DESCRIPTION, 'en', 'sv') as DESCRIPTION_SWEDISH,
  PRICE_NUM, 
  SNOWFLAKE.CORTEX.TRANSLATE(CATEGORY_1, 'en', 'fr') as CATEGORY_1_FRENCH,
  SNOWFLAKE.CORTEX.TRANSLATE(CATEGORY_2, 'en', 'de') as CATEGORY_2_GERMAN
FROM AVALANCHE.TRF.PRODUCTS_CLASSIFY;

select * from PRODUCTS_TRANSLATE;

Ever wondered how your product descriptions are perceived? Are they overwhelmingly positive, or do they carry a hint of negativity? Let's use the AI_SENTIMENT function to assign a sentiment value to each description, instantly revealing its emotional tone:

In [ ]:
CREATE OR REPLACE TABLE PRODUCTS_SENTIMENT AS
WITH CTE_SENTIMENT AS (
    SELECT
      NAME,
      DESCRIPTION,
      AI_SENTIMENT(DESCRIPTION) as SENTIMENT_JSON,
      PRICE_NUM, 
      CATEGORY_1,
      CATEGORY_2
    FROM AVALANCHE.TRF.PRODUCTS_CLASSIFY)
SELECT 
    NAME,
    DESCRIPTION,
    --SENTIMENT_JSON,
    t1.VALUE:sentiment::string as SENTIMENT,
    PRICE_NUM, 
    CATEGORY_1,
    CATEGORY_2
FROM CTE_SENTIMENT
, LATERAL FLATTEN( INPUT => SENTIMENT_JSON:categories ) t1
;

select * from PRODUCTS_SENTIMENT;

Long descriptions can be overwhelming. What if you could get the gist of a product description in a single, concise sentence? Cortex's SUMMARIZE function helps to distill lengthy text into digestible summaries.

In [ ]:
CREATE OR REPLACE TABLE PRODUCTS_SUMMARIZE AS
SELECT
  NAME,
  DESCRIPTION,
  SNOWFLAKE.CORTEX.SUMMARIZE(DESCRIPTION) as DESCRIPTION_SUMMARY,
  PRICE_NUM, 
  CATEGORY_1
  CATEGORY_2
FROM AVALANCHE.TRF.PRODUCTS_CLASSIFY;

select * from PRODUCTS_SUMMARIZE;

And sometimes you want to summarize *many rows all together*, to get the gist of what is happening *overall*. AI_SUMMARIZE_AGG is one of the aggregate AISQL functions, and it allows a quick overview.

In [ ]:
-- bonus
SELECT
  AI_SUMMARIZE_AGG(DESCRIPTION) as DESCRIPTION_SUMMARY
FROM AVALANCHE.TRF.PRODUCTS_CLASSIFY;

Sometimes, you need to pull out very specific pieces of information from unstructured text. The AI_EXTRACT function acts like a smart assistant, answering questions directly from your text. Let's use it to identify the exact product mentioned in each name, and if there are pockets.

In [ ]:
CREATE OR REPLACE TABLE PRODUCTS_EXTRACT AS
SELECT
  NAME,
  DESCRIPTION,
  AI_EXTRACT(
    text => DESCRIPTION,
    responseFormat => {'product': 'What product is being mentioned?', 'pockets': 'Are there pockets?'}) as MYANSWERS
FROM AVALANCHE.TRF.PRODUCTS_CLASSIFY;

select * from PRODUCTS_EXTRACT;

As a data engineer, or a data steward, you often want to track the data lineage of an object. The GET_LINEAGE function allows you to get the path of objects upstream or downstream of an object. The syntax is 

```sql
SNOWFLAKE.CORE.GET_LINEAGE(
    '<object_name>',
    '<object_domain>',
    '<direction>',
    [ <distance>, ]
    [ '<object_version>' ]
)
```

In [ ]:
SELECT
    DISTANCE,
    SOURCE_OBJECT_DOMAIN,
    SOURCE_OBJECT_DATABASE,
    SOURCE_OBJECT_SCHEMA,
    SOURCE_OBJECT_NAME,
    SOURCE_STATUS,
    TARGET_OBJECT_DOMAIN,
    TARGET_OBJECT_DATABASE,
    TARGET_OBJECT_SCHEMA,
    TARGET_OBJECT_NAME,
    TARGET_STATUS,
FROM TABLE (SNOWFLAKE.CORE.GET_LINEAGE(
    'AVALANCHE.TRF.PRODUCTS_SUMMARIZE', 
    'TABLE', 
    'UPSTREAM', 
    6))
ORDER BY DISTANCE;

What's the point of all this data processing if you can't easily share and interact with the insights? This is where Streamlit steps in, allowing you to transform your data into a beautiful, interactive web application with just a few lines of Python code. Let's build a simple yet powerful dashboard to visualize our product data.

In [ ]:
SQLCleanOrders.to_pandas()

In [ ]:
import streamlit as st
import altair as alt
import pandas as pd

# Ensure TOTAL_PRICE_NUM is numeric
PyTableToDataframe['TOTAL_PRICE_NUM'] = pd.to_numeric(PyTableToDataframe['TOTAL_PRICE_NUM'])

# Create the base chart with bars
chart = alt.Chart(PyTableToDataframe).mark_bar(size=15).encode(
    x=alt.X('ORDER_DATE:T',
            axis=alt.Axis(
                format='%Y-%m-%d',  # YYYY-MM-DD format
                labelAngle=90)  # Rotate labels 90 degrees
            ),
    y=alt.Y('TOTAL_PRICE_NUM:Q'),
    color=alt.condition(
        alt.datum.TOTAL_PRICE_NUM >= 500,
        alt.value('#2ecc71'),  # green for great
        alt.value('#ffff01')   # yellow for not as impressive 
    ),
    tooltip=['PRODUCT_NAME:N', 'ORDER_DATE:T'] # Add tooltip
).properties(
    height=500
)

# Display the chart
st.altair_chart(chart, use_container_width=True)